In [ ]:
import numpy as np

# 동형암호 시뮬레이터 모듈
from cipher import *
from scheme import *
from algorithms import Algorithms 
from stats import Statistics

### 동형암호 (글로벌) 파라미터 설정

In [6]:
parms = Parameters(logp = 30, logq = 150, logn = 12)
nslots = 2**parms.logn
myring = Ring(seed=1234)
context = Context(parms, myring)
print(context.params)

 logp: 30
 logq: 150 
 logn: 12


logq / logp = 150/30 = 5 이므로, 본 예제에서는 하나의 암호문은 연속으로 *5회까지만 곱셈 가능*합니다.  
그 이상은 noise가 지나치게 증가하여 'refresh'(=bootstrapping) 작업이 필요합니다.  

logq가 크면 허용되는 곱셈 깊이가 늘어나는 대신 모든 연산이 조금씩 느려지는 문제가 생기고,  
logq가 너무 작으면 아주 비싼 bootstrapping 연산을 자주 사용해야 하는 문제가 생깁니다.  
따라서 최적의 파라미터를 찾으려면 프로그램이 끝날 때 까지 **가장 깊은 곱셈이 몇 회 일어나는지** 체크할 수 있어야 합니다. 

In [7]:
# 암호 키 생성
sk = context.generate_secret_key()

keys = {"mult":context.generate_mult_key(),
        "rot":{'1':'hi1',
               '2':'hi2',
               '4':'hi4',
               '8':'hi8'}}

# 암호화, 복호화, 연산 agent
ev = Evaluator(keys, context) 
encoder = Encoder(context)
encryptor = Encryptor(context)
decryptor = Decryptor(sk)

#### 통계 연산 모듈 준비

In [68]:
algo = Algorithms(ev, encoder)
st = Statistics(ev, encoder)
check = Checker()

### variance 알고리즘 예시

#### 일반적인 버전

$$
var = \frac{\Sigma(x_i - (\bar{x}))^2}{n}
$$

In [71]:
arr = np.array([1,2,3,4,5,6,7,8])

mean = np.mean(arr)
var = np.sum((arr - mean)**2)/len(arr)
print("일반 알고리즘,", var)
print("numpy' result,", np.var(arr))

일반 알고리즘, 5.25
numpy' result, 5.25


#### 동형암호 버전

In [70]:
ctxt = encryptor.encrypt(arr)
var = st.variance(ctxt)

check.close(var._arr[0], np.var(arr))

Absolute error within: 1.0E-06
Relative error within: 1.0E-04


## 소스코드 
https://github.com/Deep-In-Sight/FHE_simulator/blob/9fb4ce2e0ea513181b7b29626de5120947e19bea/stats.py#L18

알고리즘 관련은 나머지 두 예제에서 설명 드립니다. 

```python
def variance(self, ctxt):
    ev = self.evaluator
    algo = self.algorithms

    n = algo.encode_repeat(ctxt._n_elements)
    mean = self.mean(ctxt, partial=True, duplicate=True)
    self.evaluator.rescale_next(mean)
    mean = algo.put_mask(mean, np.arange(ctxt._n_elements))
    self.evaluator.rescale_next(mean)
    sub = ev.sub(ctxt, mean)
    squared = ev.mult(sub, sub, inplace=False) # Or, square()
    self.evaluator.rescale_next(squared)

    summed_sq = algo.sum_reduce(squared, partial=True, duplicate=False)
    return ev.div_by_plain(summed_sq, n)


def mean(self, ctxt, partial=True, duplicate=True):
    ev = self.evaluator
    algo = self.algorithms

    n = algo.encode_repeat(ctxt._n_elements)
    summed = algo.sum_reduce(ctxt, partial=partial, duplicate=duplicate)
    return ev.div_by_plain(summed, n)


def sum_reduce(self,
                ctxt:CiphertextStat, 
                partial=False, 
                duplicate=False): 
    if not partial and duplicate:
        raise ValueError("Partial = False, duplicate = True not allowed.")
    ev = self.evaluator

    if partial:
        n = ctxt._n_elements
    else:
        n = ctxt.nslots
    log2n = np.log2(n).astype(int)

    # keep the original ctxt intact
    ctxt_ = ev.copy(ctxt)
    if duplicate:
        # shifted copy
        rot_copy = ev.copy(ctxt)
        ev.lrot(rot_copy, -ctxt_._n_elements)

        ev.add(ctxt_, rot_copy, inplace=True)
    for i in range(log2n):
        tmp = ev.copy(ctxt_)
        ev.lrot(tmp, 2**i, inplace=True)
        ev.add(ctxt_, tmp, inplace=True)
    return ctxt_
```

#### 동형암호 구성 


type :=  
clear text (일반문)| plain text | ciphertext 

 - 일반문을 encode하면 plain text, plain text를 encrypt하면 ciphertext가 됩니다
 - plain text 와 ciphertext는 2^n 길이의 벡터
 - ciphertext는 동형암호 scheme (backend library)에 따라 한 가지 정밀도의 int나 float으로 고정 
- **plain text**는 *nslots*, *scale* 특성을,   
- **ciphertext**는 *nslots*, *scale*, *mod* 특성을 갖습니다.   
- 암호화 후 내용물을 알 수 없으므로 부호나 정밀도에 대한 고려는 없습니다.
 
 
Op:=  (연산 결과는 모두 cipher)   
cipher-cipher mult | cipher-plain mult |  
cipher-cipher add | cipher-plain add |  
cipher-cipher sub | cipher-plain sub |  
mod_switch(cipher) | rescale(cipher) | bootstrap(cipher) |  
square(cipher) | negate(cipher) | rotate(cipher, x) 
가 있습니다.  
- backend 라이브러리마다 한 두가지 보조 연산이 추가됩니다
- cipher - clear 혹은 plain - clear 연산은 없습니다.
- 일반 자료형은 control flow에 활용됩니다.  
- ciphertext에 bitwise, comparison, 분기 관련 연산은 적용되지 않습니다. 
- division도 없습니다 